In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ContrastiveModels import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation = hard_sigmoid
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_ = 0.9999
epsilon = 0.15
one_over_epsilon = 1 / epsilon
lr_start = {'ff' : np.array([0.275, 0.1]), 'fb': np.array([0.05, 0.01])}

neural_lr_start = 0.05
neural_lr_stop = 0.001
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.01
neural_dynamic_iterations_nudged = 10
neural_dynamic_iterations_free = 20

model = ContrastiveCorInfoMax(architecture = architecture, lambda_ = lambda_, 
                              epsilon = epsilon, activation = activation)

In [5]:
_ = evaluateContrastiveCorInfoMax(model, train_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                              neural_lr_decay_multiplier, neural_dynamic_iterations_free, device)

Train accuracy :	 0.13451666666666667


In [6]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.99)**epoch_, 'fb' : lr_start['fb'] * (0.99)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        if True:
            rnd_sgn = 2*np.random.randint(2) - 1
            beta = rnd_sgn*beta
            
        neurons = model.batch_step(  x, y_one_hot, lr, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                     neural_lr_decay_multiplier, neural_dynamic_iterations_free,
                                     neural_dynamic_iterations_nudged, beta)
    
    trn_acc = evaluateContrastiveCorInfoMax(model, train_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                            neural_lr_decay_multiplier, neural_dynamic_iterations_free, device, printing = False)
    tst_acc = evaluateContrastiveCorInfoMax(model, test_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                            neural_lr_decay_multiplier, neural_dynamic_iterations_free, device, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [00:43, 68.97it/s]
7it [00:00, 69.78it/s]

Epoch : 1, Train Accuracy : 0.92325, Test Accuracy : 0.9248


3000it [00:42, 70.03it/s]
7it [00:00, 68.67it/s]

Epoch : 2, Train Accuracy : 0.9446833333333333, Test Accuracy : 0.9438


3000it [00:43, 68.28it/s]
7it [00:00, 67.99it/s]

...

Epoch : 4, Train Accuracy : 0.9631, Test Accuracy : 0.9593


3000it [00:43, 69.12it/s]
7it [00:00, 67.73it/s]

Epoch : 5, Train Accuracy : 0.96755, Test Accuracy : 0.9619


2579it [00:37, 68.86it/s]


KeyboardInterrupt: 

In [7]:
from IPython.display import Math, display
########### LATEX Style Display Matrix ###############
def display_matrix(array):
    """Display given numpy array with Latex format in Jupyter Notebook.
    Args:
        array (numpy array): Array to be displayed
    """
    data = ""
    for line in array:
        if len(line) == 1:
            data += " %.3f &" % line + r" \\\n"
            continue
        for element in line:
            data += " %.3f &" % element
        data += r" \\" + "\n"
    display(Math("\\begin{bmatrix} \n%s\\end{bmatrix}" % data))

In [8]:
display_matrix(model.B[0]['weight'][:10,:10])

<IPython.core.display.Math object>

In [10]:
display_matrix(torch.linalg.inv(model.Rh1)[:10,:10])

<IPython.core.display.Math object>

In [12]:
torch.norm(model.B[0]['weight'] - torch.linalg.inv(model.Rh1))

tensor(0.0221, device='cuda:0')

In [11]:
display_matrix(torch.linalg.inv(model.Rh2)[:10,:10])

<IPython.core.display.Math object>

In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'Contrastive CorInfoMax Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'Contrastive CorInfoMax Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
hard_sigmoid(torch.tensor(-0.1))